In [1]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
import argparse
import math
from tensorflow.keras import optimizers
from tensorflow.keras import layers, initializers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from google.cloud import storage

In [2]:
os.environ["PROJECT_ID"]="drought-watch-297217"
from google.cloud import storage
client = storage.Client()
bucket = storage.Bucket("tfrecords_data")

In [3]:
bucket

<Bucket: None>

In [4]:
print(client.project)

ludovic-giraud-290484


In [5]:
client.get_bucket('tfrecords_data')

<Bucket: tfrecords_data>

In [6]:
bucket = client.get_bucket('tfrecords_data')

In [7]:
bucket

<Bucket: tfrecords_data>

In [8]:
list(client.list_blobs(bucket))

[<Blob: tfrecords_data, data/.keep, 1606794361936555>,
 <Blob: tfrecords_data, data/packages/32b84a90e82f41bebde764ce0be9b8a369e9514bf2fd7c0090a2048e29a9af03/droughtwatch-1.0.tar.gz, 1606961340183751>,
 <Blob: tfrecords_data, data/train/_SUCCESS, 1606757932845610>,
 <Blob: tfrecords_data, data/train/packages/18a0afcaad5ebc0ee4c4b3ab8585a000861963227e202f29ccd14d497f929000/droughtwatch-1.0.tar.gz, 1606950250284447>,
 <Blob: tfrecords_data, data/train/packages/96a06f9e40ea9c8ed89e5329c05dd05515c3e0ea16f4e1d385b973590555ecf5/droughtwatch-1.0.tar.gz, 1606953999422902>,
 <Blob: tfrecords_data, data/train/packages/b55e09b3de7ecf9a6cf9821d20839f17547395b1fe3364aac2c2844624590127/droughtwatch-1.0.tar.gz, 1606954062123263>,
 <Blob: tfrecords_data, data/train/part-r-00000, 1606783796410444>,
 <Blob: tfrecords_data, data/train/part-r-00001, 1606795729376419>,
 <Blob: tfrecords_data, data/train/part-r-00002, 1606803419245464>,
 <Blob: tfrecords_data, data/train/part-r-00003, 1606803278454688>,
 <B

In [9]:
data0 = 'data/train/part-r-00000'

In [10]:
blob = bucket.get_blob(data0)

In [11]:
type(blob)

google.cloud.storage.blob.Blob

In [12]:
blob = bucket.blob(data0)
blobstring = blob.download_as_string()

In [25]:
x = blob.decode()
x[0:200]

AttributeError: 'Blob' object has no attribute 'decode'

In [39]:
import base64
with open("testfile2", "wb") as file_obj:
    #file_obj.write(base64.b64encode(blobstring))
    blob.download_to_file(file_obj)

In [22]:
type(file_obj)

_io.BufferedWriter

In [ ]:
from tensorflow.python.lib.io import file_io
file_io.FileIO(google_cloud_filename, mode='r')


In [14]:
def parse_tfrecords(filelist, batch_size, buffer_size, include_viz=False):
  # try a subset of possible bands
    def _parse_(serialized_example, keylist=['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']):
        example = tf.io.parse_single_example(serialized_example, features)

        def getband(example_key):
            img = tf.io.decode_raw(example_key, tf.uint8)
            return tf.reshape(img[:IMG_DIM**2], shape=(IMG_DIM, IMG_DIM, 1))

        bandlist = [getband(example[key]) for key in keylist]
        # combine bands into tensor
        image = tf.concat(bandlist, -1)

        # one-hot encode ground truth labels
        label = tf.cast(example['label'], tf.int32)
        label = tf.one_hot(label, NUM_CLASSES)

        return {'image': image}, label

    tfrecord_dataset = tf.data.TFRecordDataset(filelist)
    tfrecord_dataset = tfrecord_dataset.map(lambda x:_parse_(x)).shuffle(buffer_size).repeat(-1).batch(batch_size)
    tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()
    image, label = tfrecord_iterator.get_next()
    return image, label

In [15]:
def file_list_from_folder(folder, data_path):
  folderpath = os.path.join(data_path, folder)
  filelist = []
  for filename in os.listdir(folderpath):
      if filename.startswith('part-') and not filename.endswith('gstmp'):
          filelist.append(os.path.join(folderpath, filename))
  return filelist

In [16]:
file_obj.

<_io.BufferedWriter name='testfile'>

In [17]:
filelist = [file_obj]

In [20]:
features = {
  'B1': tf.io.FixedLenFeature([], tf.string),
  'B2': tf.io.FixedLenFeature([], tf.string),
  'B3': tf.io.FixedLenFeature([], tf.string),
  'B4': tf.io.FixedLenFeature([], tf.string),
  'B5': tf.io.FixedLenFeature([], tf.string),
  'B6': tf.io.FixedLenFeature([], tf.string),
  'B7': tf.io.FixedLenFeature([], tf.string),
  'B8': tf.io.FixedLenFeature([], tf.string),
  'B9': tf.io.FixedLenFeature([], tf.string),
  'B10': tf.io.FixedLenFeature([], tf.string),
  'B11': tf.io.FixedLenFeature([], tf.string),
  'label': tf.io.FixedLenFeature([], tf.int64),
}

In [34]:
IMG_DIM = 65
NUM_CLASSES = 4

In [40]:
parse_tfrecords(["testfile2"],100,100)

({'image': <tf.Tensor: shape=(100, 65, 65, 7), dtype=uint8, numpy=
  array([[[[ 39,  36,  38, ...,  51,  38,  36],
           [ 38,  35,  36, ...,  49,  36,  36],
           [ 36,  33,  34, ...,  50,  35,  34],
           ...,
           [ 42,  40,  43, ...,  66,  47,  41],
           [ 41,  39,  40, ...,  65,  46,  39],
           [ 41,  39,  41, ...,  65,  46,  39]],
  
          [[ 36,  32,  33, ...,  46,  34,  35],
           [ 35,  31,  32, ...,  45,  32,  36],
           [ 34,  30,  30, ...,  41,  30,  33],
           ...,
           [ 43,  41,  44, ...,  70,  50,  41],
           [ 42,  40,  44, ...,  70,  50,  43],
           [ 41,  39,  43, ...,  67,  48,  40]],
  
          [[ 34,  29,  30, ...,  39,  29,  32],
           [ 34,  29,  29, ...,  39,  28,  31],
           [ 33,  29,  29, ...,  38,  28,  30],
           ...,
           [ 43,  41,  45, ...,  71,  50,  42],
           [ 43,  42,  48, ...,  76,  55,  43],
           [ 41,  40,  45, ...,  73,  54,  43]],
  
         